# Data Description: NYC Taxi & Limousine Commission Data
This street cube is created using 2011 December Hangzhou data.  This section captures specifics on the data stored in the database that was used to create the TrajCube.

## Setup Logic
This section is just setting up the code to query the database and format the results so that subsequent sections can be more concise.

In [1]:
# Getting a connection via the jango settings.py
from django.db import connection

cursor = connection.cursor()

In [2]:
from datetime import datetime
from IPython.display import Markdown
from itertools import islice
from collections import namedtuple
from entity.loader.taxi.taxi_common import human_size

def format_results(results, limit=None):
    try:
        if len(results) > 1:
            sep = '-:'
            nl = '\n'
            col_sep = '|'
            if limit:
                results = results[:limit]
        else:
            sep = ' = '
            nl = ''
            col_sep = ', '
        return Markdown(nl.join((
            col_sep.join(  # Header row
                [f for f in results[0]._fields]
            ),
            col_sep.join(  # Alignment Row
                ['-:'] * len(results[0]._fields)
            ) if len(results) > 1 else ' = ',
            '\n'.join(  # Data Rows
                [col_sep.join([str(getattr(r, f)) for f in r._fields]) for r in results]
            ).rstrip()
        )))
    except BaseException:
        return results

num_results = 20  # Default maximum results to print from a query

def execute(q_str, max_results=num_results, execute_timeit=True):
    if execute_timeit:
        if not q == q_str:  # %timeit requires using the global q...
            raise Exception(
                "Set global 'q' before calling execute.  Would time {} instead".format(q)
            )
        %timeit cursor.execute(q)
    
    cursor.execute(q_str)
    nt_result = namedtuple('Result', [col[0] for col in cursor.description])
    return [nt_result(*r) for r in islice(cursor, None, max_results)]

## Input Data Statistics
The purpose of this section is to see what kind of data is actually in the base data.  This gives an idea about the total amount of data that the street cube is dealing with.

In [3]:
from datetime import datetime
from entity.models import Trip
from entity.functions import NumPoints

start_time = datetime.now()
qs = (Trip.objects
      .datetimes(field_name='start_datetime', kind='day')
      .values('datetimefield')
      .annotate(num_trips=Count('id'))
      .annotate(num_samples=Sum(NumPoints('geometry')))
#       .order_by('start_datetime')
)

total_trips = total_samples = 0
for result in qs:
    total_trips += result['num_trips']
    if result['num_samples']:  # TODO: Why is this None?
        samples = result['num_samples']
    else:
        samples = 2 * result['num_trips']
    total_samples += samples
#     print('{}: Trips({:8,}), Samples({:9,})'.format(
#             result['datetimefield'].date(), result['num_trips'], samples
#     ))
print('Total {:,} trips over {:,} samples.'.format(total_trips, total_samples))
print('Query execution and results in {}'.format(datetime.now() - start_time))

Total 29,284,418 trips over 524,541,298 samples.
Query execution and results in 0:03:48.667259


In [4]:
# Time range of the loaded data
q = """
    SELECT min(start_datetime) as min_timestamp, max(start_datetime) as max_timestamp
    FROM entity_trip
""".replace('\n', ' ')
format_results(execute(q))

The slowest run took 588.61 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 108 µs per loop


min_timestamp, max_timestamp = 2005-01-09 02:31:37+00:00, 2080-01-12 11:07:23+00:00

# Cube Statistics
Having built the cube it's useful to know how much data was processed to build the cube and what kind of size each dimension has in the cube.  The queries below provide some useful statistics related to the cube that was created above.

## Counts for each dimension

In [5]:
# Count of the total non-empty cells in the cube.
q = """
    SELECT count(*) as nonempty_cube_cells
    FROM streetcube_origindestinationcell
    WHERE
        time_inc IS NOT NULL
        AND
        osm_id IS NOT NULL
"""
results = execute(q)
nonempty_cube_cells = results[0].nonempty_cube_cells
format_results(results)

The slowest run took 5.06 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 87.4 µs per loop


nonempty_cube_cells = 0

### Street segments (unique across pickup and dropoff)
The structure of the cube stores the street identifier as a node and then stores two lists representing the pickups from and dropoffs to that street.  So there are basically three metrics related to each row.

1. The number of unique streets (combined set of pickup and dropoff)
1. The number of streets where pickups occur
1. The number of streets where dropoffs occur

In [6]:
# Number of unique street segments
q = """
    SELECT count(distinct osm_id) as count_origins
    FROM streetcube_origindestinationcell
"""
results = execute(q)
unique_streets = results[0].count_origins
format_results(results)

The slowest run took 4.09 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 78.1 µs per loop


count_origins = 0

In [7]:
# Unique streets for pickups and dropoffs.
q = """
SELECT (
    SELECT count(DISTINCT osm_id)
    FROM streetcube_origindestinationcell
    WHERE
        measures ? 'dropoff'
        AND
        time_inc IS NULL
        AND
        osm_id IS NOT NULL
) as unique_pickup_locations,
(
    SELECT count(DISTINCT osm_id)
    FROM streetcube_origindestinationcell
    WHERE
        measures ? 'pickup'
        AND
        time_inc IS NULL
        AND
        osm_id IS NOT NULL
) as unique_dropoff_locations
""".replace('\n', ' ')
format_results(execute(q))

The slowest run took 90.15 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 127 µs per loop


unique_pickup_locations, unique_dropoff_locations = 0, 0

### Unique time slots

In [8]:
# Number of unique time segments in the cube
q = """
    SELECT count(distinct time_inc) as count_times
    FROM streetcube_origindestinationcell
"""
results = execute(q)
unique_times = results[0].count_times
format_results(results)

The slowest run took 5.44 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 74.4 µs per loop


count_times = 0

### Compare the Dense vs. Sparse Matrix sizes
This is intended to give insight into whether the sparse matrix approach is actually saving any space by storing the dimension values as a list instead of hashing into a true 3D array.

In [9]:
# Get insight on total savings from a dense representation like imMens uses.
size_of_time = 8  # bytes
size_of_osm = 8  # bytes
size_of_record = 8  # bytes
dense_matrix_size = unique_streets * unique_streets * unique_times * size_of_record
sparse_matrix_size = nonempty_cube_cells * (
    size_of_time + size_of_osm + size_of_osm + size_of_record
)
'Estimated {} to store sparse and {} for dense, saving {}.'.format(
    human_size(sparse_matrix_size),
    human_size(dense_matrix_size),
    human_size(dense_matrix_size-sparse_matrix_size)
)

'Estimated 0.0 to store sparse and 0.0 for dense, saving 0.0.'

## Cube Size on Disk

In [10]:
# Get the size on disk of the cube table.
q = """
SELECT pg_size_pretty(sum(
    CAST(pg_relation_size(C.oid) AS BIGINT)
)) AS "total_db_size"
FROM pg_class C
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace)
WHERE nspname NOT IN ('pg_catalog', 'information_schema')
"""
results = execute(q, execute_timeit=False)
'Total DB including entity data is {} on disk.'.format(results[0].total_db_size)

'Total DB including entity data is 153 GB on disk.'

In [11]:
# Get the size on disk of the cube table.
q = """
    SELECT pg_size_pretty(
        pg_total_relation_size('streetcube_origindestinationcell')
    ) AS traj_cube_size__od
"""
results = execute(q, execute_timeit=False)
'TrajCube::OD is size {} on disk.'.format(results[0].traj_cube_size__od)

'TrajCube::OD is size 32 kB on disk.'

# Queries
There are some general queries that are identified as common tasks for the system:

1. Find the Top N streets for pickup/dropoff in a given time and/or area
1. Find pickup locations with diverse dropoff locations (i.e. what location goes to the most different dropoff locations)
1. Find pickup locations with common dropoff (i.e. same pickup and dropoff locations)
1. Calculate aggregations across streets in a given time and/or area

### Find Top N streets for pickup, dropoff, and both
There is a desire to identify locations and times where there are many people being dropped off and picked up in the same location.  Relaxing the query below can also individually find areas with only high dropoffs or areas with high pickups independently.

In [12]:
q = """
    SELECT
        time_inc,
        osm_id,
        (measures->'pickup'->>'count')::int as pickups,
        (measures->'dropoff'->>'count')::int as dropoffs
    FROM streetcube_origindestinationcell
    WHERE 
        measures ?& array['pickup', 'dropoff']
        AND time_inc IS NOT NULL AND osm_id IS NOT NULL
        AND (measures->'pickup'->>'count')::int > 100
        AND (measures->'dropoff'->>'count')::int > 100
    ORDER BY pickups DESC, dropoffs DESC
    LIMIT 5
""".replace('\n', ' ')
format_results(execute(q))

The slowest run took 191.89 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 140 µs per loop


[]

### Find areas where people travel to disperse locations
Some areas that have a high degree of pickups might have different profiles.  Most likely each taxi travels to a different destination.  This query finds the pickup locations that have the most diverse set of dropoff locations.

In [13]:
q = """
    WITH counts AS (
        WITH expanded AS (
            SELECT
                id,
                time_inc,
                jsonb_object_keys(measures->'dropoff'->'items') as keys
                FROM streetcube_origindestinationcell
                WHERE measures ? 'dropoff'
                AND time_inc IS NOT NULL AND osm_id is NOT NULL
        )
        SELECT
            id,
            time_inc,
            count(DISTINCT keys) as unique_dropoff_count
        FROM expanded
        GROUP BY id, time_inc
    )
    SELECT * 
    FROM counts
    WHERE unique_dropoff_count > 90
    ORDER BY unique_dropoff_count DESC
    LIMIT 5
""".replace('\n', ' ')
results = execute(q)
format_results(results)

The slowest run took 4.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 163 µs per loop


[]

### Find areas that travel to common locations
Another mobility pattern is when a lot of trip happen between common locations.  This is interesting since it identifies a connection between the two locations.  The resulting query identifies the two locations with the most trips between them.

In [14]:
q = """
    WITH counts AS (
        SELECT
            id as pickup_id,
            osm_id,
            time_inc,
            d.key::int as dropoff_id,
            d.value::text::int as trip_count
        FROM
            streetcube_origindestinationcell,
            jsonb_each(measures->'dropoff'->'items') as d
        WHERE measures ? 'dropoff'
        AND time_inc IS NOT NULL AND osm_id is NOT NULL    
    )
    SELECT *
    FROM counts
    WHERE trip_count > 5
    AND dropoff_id != pickup_id
    ORDER BY trip_count
    LIMIT 5
""".replace('\n', ' ')
results = execute(q)
pickup_osm_list = tuple(str(r.osm_id) for r in results)
format_results(results)

The slowest run took 213.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 165 µs per loop


[]

# Linking Data
In the origin-destination cube the data needs to be linked so there is a need to find data related to the start and end points of each trip.

### Pickup Perspective: Find the dropoffs locations when provided pickup streets
This is the table from the perspective of pickup locations.  That is, the input list represents a set of street identifiers, such as could be selected graphically, and the goal is to identify the linked dropoff locations.

In [15]:
q = ("""
    WITH pickup_base AS (
        SELECT
            pickup.id as pickup_id,
            pickup.osm_id as pickup_street,
            pickup.time_inc as pickup_time,
            items.key::int as dropoff_id,
            items.value::text::int as dropoff_count
        FROM
            streetcube_origindestinationcell as pickup,
            jsonb_each(pickup.measures->'dropoff'->'items') as items
        WHERE
            pickup.measures ? 'dropoff'
            AND
            pickup.osm_id IN %s
            AND time_inc IS NOT NULL AND osm_id is NOT NULL
    )
    SELECT
        pickup_street,
        pickup_time,
        dropoff.osm_id as dropoff_street,
        dropoff.time_inc as dropoff_time,
        dropoff_count
    FROM
        pickup_base,
        streetcube_origindestinationcell as dropoff
    WHERE
        dropoff.id = dropoff_id
    ORDER BY dropoff_count DESC
    LIMIT 5
""" % str(pickup_osm_list)).replace('\n', ' ')
results = execute(q)
format_results(results)

ProgrammingError: syntax error at or near ")"
LINE 1: ...ff'             AND             pickup.osm_id IN ()         ...
                                                             ^


### Dropoff Perspective: Find pickup locations related to provided dropoff locations
This is the same as the list above except that it does the query by finding where people are coming from in the given locations.

In [ ]:
q = ("""
    WITH dropoff_base AS (
        SELECT
            dropoff.id as dropoff_id,
            dropoff.osm_id as dropoff_street,
            dropoff.time_inc as dropoff_time,
            items.key::int as pickup_id,
            items.value::text::int as pickup_count
        FROM
            streetcube_origindestinationcell as dropoff,
            jsonb_each(dropoff.measures->'pickup'->'items') as items
        WHERE
            dropoff.osm_id IN {}
            AND
            dropoff.measures ? 'pickup'
            AND time_inc IS NOT NULL AND osm_id is NOT NULL
    )
    SELECT
        dropoff_street,
        dropoff_time,
        pickup.osm_id as pickup_street,
        pickup.time_inc as pickup_time,
        pickup_count
    FROM
        dropoff_base,
        streetcube_origindestinationcell as pickup
    WHERE
        pickup.id = pickup_id
    ORDER BY pickup_count DESC
    LIMIT 5
""".format(pickup_osm_list)).replace('\n', ' ')
results = execute(q)
format_results(results)

# Aggregations Across Dimensions
This section provides the queries that aggregate across a single dimension.  These will be used to save the aggregation in the cube for faster recall later when that dimension is unconstrained in the query.

The supported queries include:
1. Aggregate in the time dimension
1. Aggregate in the general street psuedo-dimension
1. Aggregate in the pickup dimension
1. Aggregate in the dropoff dimension

## Unbound time aggregation
This is where the time dimension is not bound, meaning that all records must be analyzed regardless of time.  For example, which street has the most pickups or dropoffs.

In [ ]:
q = """
    WITH expanded_measures AS (
        SELECT 
            osm_id,
            m.key,
            (m.value->>'count')::bigint as count
        FROM streetcube_origindestinationcell, jsonb_each(measures) as m
        WHERE time_inc IS NULL and osm_id IS NOT NULL
    )
    SELECT *
    FROM expanded_measures
    ORDER BY count DESC
    LIMIT 6
""".format(pickup_osm_list)
results = execute(q)
format_results(results)

## Unbound street aggregation
This is where the street dimension is not bound and all records related must be combined.  This aggregation can be used to answer questions such as:
1. In this set of times, which street has the most pickups/dropoffs
1. Which times have the most pickups/dropoffs?  What are the pickup/dropoff streets?

In [ ]:
q = """
    WITH expanded_measures AS (
        SELECT 
            time_inc,
            m.key,
            (m.value->>'count')::bigint as count
        FROM streetcube_origindestinationcell, jsonb_each(measures) as m
        WHERE time_inc IS NOT NULL and osm_id IS NULL
    )
    SELECT *
    FROM expanded_measures
    ORDER BY count DESC
    LIMIT 6
""".format(pickup_osm_list)
results = execute(q)
format_results(results)

## Rollup along pickup/dropoff locations and time
Each (time, street) pair provides a list of pickup and dropoff locations associated with it.  Along with that list is a pre-calculated measure that is the aggregation across the entire list.  This aggregation is seen in many of the previous queries where the returned lists have structures such as:
<pre>
{
    'dropoff': {
       'items': { ... },
       'count': 24
    },
    'pickup': {
       'items': { ... },
       'count': 17
    }
}
</pre>
In the example above, from the street and time referenced by this record, there are 24 outgoing edges (i.e. 24 trips from this street to other streets.  There are 17 incoming edges, meaning that 17 trips started somewhere and complete at this street.  The effect of having this pre-aggregation can be observed when the desired query does not require specifics about both ends of the trip, such as when simply trying to summarize the number of trips leaving a particular street as in the examples below that quantify performing the query with and without the pre-aggregation.

### Not using the pre-rollup aggregation

In [ ]:
# Rollup along the pickup locations
q = ("""
    WITH pickup_base AS (
        SELECT
            pickup.id as pickup_id,
            pickup.osm_id as pickup_street,
            pickup.time_inc as pickup_time,
            item.key::int as dropoff_id,
            item.value::text::int as dropoff_count
        FROM
            streetcube_origindestinationcell as pickup,
            jsonb_each(pickup.measures->'dropoff'->'items') as item
        WHERE
            pickup.osm_id IN %s
            AND pickup.time_inc IS NOT NULL
            AND pickup.measures ? 'dropoff'
    )
    SELECT
        pickup_street,
        sum(dropoff_count) as dropoff_count
    FROM
        pickup_base,
        streetcube_origindestinationcell as dropoff
    WHERE
        dropoff.id = dropoff_id
    GROUP BY pickup_street
    ORDER BY dropoff_count DESC
""" % str(pickup_osm_list)).replace('\n', ' ')
results = execute(q)
format_results(results)

### Using pre-rollup aggregation data

In [ ]:
# Rollup along the pickup locations.  This finds the same answer as above but much faster.
q = ("""
    SELECT
        pickup.osm_id as pickup_street,
        sum((pickup.measures->'dropoff'->>'count')::int) as dropoff_count
    FROM
        streetcube_origindestinationcell as pickup
    WHERE
        pickup.osm_id IN %s
        AND pickup.measures ? 'dropoff'
        AND pickup.time_inc IS NOT NULL
    GROUP BY pickup_street
    ORDER BY dropoff_count DESC
""" % str(pickup_osm_list)).replace('\n', ' ')
results = execute(q)
format_results(results)

In [ ]:
# Just getting the total number of pickup and dropoff.  Note that this can also provide the totals
# for individual streets so that getting the total pickups or dropoffs
q = """
    SELECT
        m.key as measure_label,
        (m.value->>'count')::bigint as count
    FROM
        streetcube_origindestinationcell as od,
        jsonb_each(measures) as m
    WHERE
        osm_id IS NULL AND time_inc IS NULL
""".replace('\n', ' ')
results = execute(q)
format_results(results)

# Other Queries
## Organizing data by month

In [ ]:
q = ("""
    WITH agg AS (
        WITH time_base AS (
            WITH expanded_items AS (
                WITH expanded_measures AS (
                    SELECT
                        time_inc,
                        m.key as measure_label,
                        (m.value->'items') as items
                    FROM
                        streetcube_origindestinationcell,
                        jsonb_each(measures) as m
                    WHERE osm_id IS NULL AND time_inc IS NOT NULL
                )
                SELECT
                    time_inc,
                    measure_label,
                        item.key as item_id,
                        item.value::text::int as item_count
                    FROM
                        expanded_measures,
                        jsonb_each(items) as item
            )
            SELECT
                time_inc,
                extract(DAY FROM time_inc) as day_of_month,
                extract(ISODOW FROM time_inc) as day_of_week,
                extract(DOY FROM time_inc) as day_of_year,
                extract(ISOYEAR FROM time_inc) as year,
                extract(MONTH FROM time_inc) as month,
                extract(WEEK FROM time_inc) as week,
                extract(HOUR FROM time_inc) as hour,
                (SELECT extract(ISODOW FROM time_inc) <=5) as workday,
                measure_label,
                item_id,
                item_count
            FROM expanded_items
        )
        SELECT
            month,
            measure_label,
            sum(item_count) as total_count
        FROM time_base
        GROUP BY month, measure_label
    )
    SELECT
        month::int,
        json_object_agg(
            measure_label, total_count
        ) as measures
    FROM agg
    GROUP BY month
    ORDER BY month
""").replace('\n', ' ')
results = execute(q)
format_results(results)